<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Environment</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main</a></span><ul class="toc-item"><li><span><a href="#Original-Cpp" data-toc-modified-id="Original-Cpp-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Original Cpp</a></span></li><li><span><a href="#New-Cpp" data-toc-modified-id="New-Cpp-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>New Cpp</a></span></li><li><span><a href="#Test-PyInteractions" data-toc-modified-id="Test-PyInteractions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Test <code>PyInteractions</code></a></span></li></ul></li></ul></div>

# Preamble

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from simfection.population_engine import PopulationEngine
from simfection.settings import SimfectionSettings
from simfection.connection_engine import ConnectionEngine
from simfection.interaction_engine_cpp import InteractionEngineCpp
from simfection.interaction_engine_cpp import InteractionEngineCpp2
from network import PyInteractions

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


2020-06-28 09:30:17,777  matplotlib.pyplot               DEBUG   Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Environment

In [2]:
settings = SimfectionSettings()
population_engine = PopulationEngine(settings)
population_engine.make_dummy()

connections = ConnectionEngine(
    population = population_engine._df,
    settings = settings
)
connections.create_connections()

2020-06-28 09:30:17,794  simfection.population_engine    DEBUG   + Initializing population engine.
2020-06-28 09:30:17,798  simfection.population_engine    DEBUG   - Unloading settings.
2020-06-28 09:30:17,803  simfection.population_engine    INFO    + Synthesizing a dummy population.
2020-06-28 09:30:17,849  numexpr.utils                   INFO    NumExpr defaulting to 4 threads.
2020-06-28 09:30:17,888  simfection.population_engine    DEBUG   - Population states: {'inf': 0.38, 'sus': 0.36, 'dead': 0.26}
2020-06-28 09:30:17,889  simfection.connection_engine    DEBUG   + Initializing connection engine.
2020-06-28 09:30:17,890  simfection.connection_engine    DEBUG   - Creating connections.
2020-06-28 09:30:19,268  simfection.connection_engine    DEBUG   - All connections made successfully.


# Main

## Original Cpp

In [3]:
interactions = InteractionEngineCpp(
    connections=connections.connections,
    population=connections.population,
    settings=settings
)

2020-06-28 09:30:19,275  simfection.interaction_engine_cpp  DEBUG   + Initializing interaction engine.


We call `_init_interactions` first so we can pull out the `pop_immunity` for teting.`

In [4]:
interactions._init_interactions()

In [5]:
pop_immunity = interactions.population['immunity'].tolist()

In [6]:
interactions.interact_all()

2020-06-28 09:30:19,314  simfection.interaction_engine_cpp  DEBUG   - Running InteractionEngine with C++ optimizations.


## New Cpp

In [7]:
interactions2 = InteractionEngineCpp2(
    connections=connections.connections,
    population=connections.population,
    settings=settings
)

2020-06-28 09:30:19,346  simfection.interaction_engine_cpp  DEBUG   + Initializing interaction engine.


Check that the immunity objects produced by the origina cpp version (i.e. `pop_imm`) matches the new immunity object (i.e. `_population.get('immunity')`).

In [8]:
_population, _connections, _pathogen = interactions2._prep_for_cpp()

In [9]:
_population.get('immunity') == pop_immunity

True

In [10]:
interactions2._set_interactions(
    _population,
    _connections,
    _pathogen
)

TypeError: expected bytes, list found

## Test `PyInteractions`

In [11]:
py_interactions = PyInteractions()

In [12]:
py_interactions.set_population(
    _population.get('agent'),
    _population.get('state'),
    _population.get('infected_by'),
    _population.get('days_infected'),
    pop_immunity,
)

TypeError: expected bytes, list found

Something weird is happening. It works when I just do `InteractionEngineCpp.interact_all()`, but not when I try to just work with `PyInteractions`. 